# Housing Project

## Setting up

In [4]:
import sys  
sys.path.insert(0, r'C:\Users\Cora\OneDrive\ML-Preparation\HousingProject')

In [8]:
import pandas as panda
import numpy as numpy

from functions import fetch_housing_data

## Download Data

In [9]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "C:/Users/Cora/OneDrive/ML-Preparation/HousingProject/dataset/"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"
fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH)

tst
Input Data URL source and target dir.


In [3]:
from functions import tst_fun